In [ ]:
import matplotlib
import pathlib
import mne
from mne.time_frequency import tfr_morlet
from mne.channels import find_ch_adjacency
import os

Ensure Matplotlib uses the Qt5Agg backend, which is the best choice for MNE-Python's interactive plotting functions.

In [ ]:
matplotlib.use('Qt5Agg')

In [ ]:
import matplotlib.pyplot as plt

Define paths, load and read data

In [ ]:
data_folder = '/Users/christinadelta/datasets/emotions_task'
raw_file = os.path.join(data_folder, 'emotion_rec-fif.gz')

raw = mne.io.read_raw_fif(raw_file, preload=True)
#raw = mne.io.read_raw_fif(raw_file)

In [ ]:
raw.plot()

In [ ]:
raw.info

Filter the data

In [ ]:
raw.filter(1,20)

In [ ]:
# visualise both
raw.plot(title='raw')
raw_filt.plot(title='filtered')
raw_filt2.plot(title='filtered2')

### Run ICA 

In [ ]:
ica = mne.preprocessing.ICA(n_components=20, random_state=0)

In [ ]:
ica.fit(raw.copy().filter(8,35))

In [ ]:
ica.plot_components()

Remove the noisy components using the ica exclude. I have visualy inspected the components and marked the bad ones. So I need to only type ```ica.exclude```. If I had not marked the bad components I used specify a list like: ```ica.exclude = [components to be excluded]```

Try detecting the bad components with the automatic algorithm and see the differences 

In [ ]:
bad_idx, scores = ica.find_bads_eog(raw, 'SO2', threshold=2)
print(bad_idx)

In [ ]:
ica.exclude = bad_idx

In [ ]:
ica.apply(raw.copy(), exclude=ica.exclude).plot()

### Find Events

In [ ]:
events = mne.find_events(raw)
events

In [ ]:
event_id = {
    "type_1": 200,
    "type_2": 100
}

In [ ]:
raw.plot(events = events, event_id = event_id)

In [ ]:
epochs = mne.Epochs(raw, 
                    events, 
                    event_id=event_id,
                    preload=True)

In [ ]:
epochs.plot(events=events, n_epochs=15)

### Apply ICA on epochs

In [ ]:
epochs_ica = ica.apply(epochs, exclude=ica.exclude)

In [ ]:
epochs.apply_baseline((None, 0))

In [ ]:
# look at subsets of epochs 
epochs["type_1"]

In [ ]:
epochs.info

### How does the epoched activity look like?

In [ ]:
epochs['type_2'].plot_image(picks=[13])

### Equalize the happy-face epochs and the fear-face epochs 

In [ ]:
epochs.equalize_event_counts(event_id)
epochs

### Save the epochs object for later use (if needed)

In [ ]:
epochs.save(os.path.join(data_folder, 'emotion_rec-epo.fif'), overwrite=True) 

### Create epochs for time-frequency analysis

In [ ]:
tmin = -.5
tmax = 1.5
tfr_epochs = mne.Epochs(raw, 
                        events, 
                        event_id=event_id, 
                        tmin=tmin, 
                        tmax=tmax, 
                        preload=True)

tfr_epochs = ica.apply(tfr_epochs, exclude=ica.exclude)

In [ ]:
tfr_epochs.equalize_event_counts(event_id)

In [ ]:
tfr_epochs.save(os.path.join(data_folder, 'emotion_rec_tfr-epo.fif'), overwrite=True) 

### Evoked 
average epochs over trials 

In [ ]:
type1 = epochs['type_1'].average()
type1

type2 = epochs['type_2'].average()
type2

In [ ]:
epochs

To take a look at the actual data 

In [ ]:
epochs.get_data().shape

In [ ]:
type1.plot_topomap(times = [0.1, 0.2, 0.3, 0.4])

In [ ]:
type2.plot_topomap(times = [0.1, 0.2, 0.3, 0.4])

In [ ]:
type2.plot()

In [ ]:
type1.plot()

In [ ]:
type2.plot_joint(times = [0.1, 0.2, 0.3, 0.4])

### Plot condition contrasts

In [ ]:
diff = mne.combine_evoked((type2, -type1), weights='equal')
diff.plot_joint()

### Plot condition contrasts as an image

In [ ]:
diff.plot_image()

In [ ]:
rois = mne.channels.make_1020_channel_selections(diff.info, midline="z12")
diff.plot_image(group_by=rois, show=False, show_names="all")

In [ ]:
mne.viz.plot_compare_evokeds({"fear": type1,
                              "happy": type2}, picks=[13])

In [ ]:
type2.plot_topo()

### Time-Frequency Analysis

In [ ]:
happy = type2.data
fear = type1.data
ch_names = type2.info['ch_names']

In [ ]:
samp_ch = 'C4'
ch_idx = [i for i, j in enumerate(ch_names) if j == samp_ch]

In [ ]:
plt.plot(happy[ch_idx[0],:])
plt.title(samp_ch)
plt.ylabel('amplitude')
plt.xlabel('time')

In [ ]:
epochs_tfr = tfr_epochs
epochs_tfr

In [ ]:
epochs_tfr.plot_psd(fmin=0, fmax=40)

### What aboout effects?
We will extract power per time and frequency with Morlet wavelets 

In [ ]:
freqs = list(range(3,30))
tfr_happy = tfr_morlet(epochs_tfr["type_2"], freqs, 3, return_itc=False)
tfr_fear = tfr_morlet(epochs_tfr["type_1"], freqs, 3, return_itc=False)

In [ ]:
tfr_happy.data.shape

Now, the time-frequency data is stored in TRF objects 

In [ ]:
tfr_contrast = mne.combine_evoked((tfr_fear, tfr_happy), (-.5, .5))
tfr_contrast.apply_baseline((None, 0))

when in time and at what frequencies do we have significant differencies between the two conditions?

In [ ]:
tfr_contrast.plot_joint()

### Run statistics
Can we statistically threshold the images to see which effects are reliable?

In [ ]:
diff.plot_image(group_by=rois, show=False, show_names="all")

### Cluster-based permutation stats


In [ ]:
adjacency, ch_names = find_ch_adjacency(epochs.info, ch_type='eeg')
plt.imshow(adjacency.toarray(), cmap="Greys")

Now we need the data in the right shape. Sadly, because the space dimension needs to be last, we need to manually swap the time and space axes.

In [ ]:
epochs.pick_types(eeg=True)
type2_epochs, type1_epochs = epochs["type_2"].get_data(), epochs["type_1"].get_data()
type2_epochs.shape, type1_epochs.shape

In [ ]:
type2_epochs = type2_epochs.swapaxes(1, 2)
type1_epochs = type1_epochs.swapaxes(1, 2)
type2_epochs.shape, type1_epochs.shape